In [1]:
import graphlab
import numpy as np
import pandas as pd
import matplotlib 

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
# Drop the duplicates 
Total_Test = pd.read_csv("../Total_Test.csv")
Total_Test = Total_Test.drop_duplicates()
Total_Test['question_id'] = Total_Test['question_id'].astype(str)
Total_Test.to_csv('../Total_Test.csv', encoding='utf-8', index = False)

/home/zpgao/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1,2,3,4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Total_Test = pd.read_csv("../Total_Test.csv")

In [4]:
def Format_Voted():
    answer_voted = Total_Test[['question_id','question_content','question_tags','answer_voted' ]]
    answer_voted.to_csv('./Answer_Voted.csv', encoding='utf-8', index = False)
    
    voted = graphlab.SFrame.read_csv('./Answer_Voted.csv')
    been_voted =  voted[ voted['answer_voted'] > 0 ] 
    un_voted = voted[ voted['answer_voted'] == 0 ]
    
    # been_voted = been_voted.remove_column('answer_voted') # 18134
    # un_voted = un_voted.remove_column('answer_voted')  # 18123
    
    # No drop duplicates save as 'been_voted.csv' & 'un_voted.csv' 
    been_voted.save('./been_voted.csv', format='csv')
    un_voted.save('./un_voted.csv', format='csv')
    
    # Dorp duplicates save as 'Been_Voted.csv' & 'Un_Voted.csv'
    Been_Voted = pd.read_csv('./been_voted.csv')
    Un_Voted = pd.read_csv('./un_voted.csv')
    
    Been_Voted = Been_Voted.drop('answer_voted', 1)
    Un_Voted = Un_Voted.drop('answer_voted', 1)
    Been_Voted = Been_Voted.drop_duplicates()
    Un_Voted = Un_Voted.drop_duplicates()
    Been_Voted.to_csv('./Been_voted.csv', encoding='utf-8', index = False )
    Un_Voted.to_csv('./Un_voted.csv', encoding='utf-8', index = False )

In [5]:
Format_Voted()

[INFO] 1460455135 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /home/zpgao/anaconda2/lib/python2.7/site-packages/certifi/cacert.pem
1460455135 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to beyond_acm@163.com and will expire on January 04, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-5242 - Server binary: /home/zpgao/anaconda2/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1460455135.log
[INFO] GraphLab Server Version: 1.7.1


PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK/SIMILAR/Answer_Voted.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.157899 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Unable to parse line "question_id,question_content,question_tags,answer_voted"
PROGRESS: 1 lines failed to parse correctly
PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK/SIMILAR/Answer_Voted.csv
PROGRESS: Parsing completed. Parsed 39568 lines in 0.087875 secs.


In [6]:
# Total_Test

In [7]:
question_content = Total_Test[['question_id','question_content','question_tags']]
question_content = question_content.drop_duplicates()
question_content.to_csv('../Question_Content.csv', encoding='utf-8', index = False)

In [8]:
question_content = graphlab.SFrame.read_csv('../Question_Content.csv')

PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK/Question_Content.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.195528 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Unable to parse line "question_id,question_content,question_tags"
PROGRESS: 1 lines failed to parse correctly
PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK/Question_Content.csv
PROGRESS: Parsing completed. Parsed 31012 lines in 0.070604 secs.


In [9]:
# question_content   # 31012 x 3

In [10]:
# question_content

In [11]:
# best_answer[0]
# question_content = question_content.remove_column('answer_voted')
question_content['text'] = question_content['question_tags'] + question_content['question_content']
question_content['word_count'] = graphlab.text_analytics.count_words(question_content['text'])

In [12]:
# len( question_content )     # 36257
# question_content.unique()

In [13]:
tfidf = graphlab.text_analytics.tf_idf(question_content['word_count'])

In [14]:
# tfidf

In [15]:
question_content['tfidf'] = tfidf

In [16]:
# question_content  # 31012 x 6

In [17]:
been_voted = graphlab.SFrame.read_csv('./Been_voted.csv')
un_voted = graphlab.SFrame.read_csv('./Un_voted.csv')

PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK/SIMILAR/Been_voted.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.144733 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK/SIMILAR/Been_voted.csv
PROGRESS: Parsing completed. Parsed 16358 lines in 0.040058 secs.
PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK/SIMILAR/Un_voted.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.071741 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type l

In [18]:
# len(been_voted)
# been_voted

In [19]:
# sid = been_voted[100]['question_id']
# question_content[ question_content['question_id'] == sid ]

In [20]:
def Select_Question( num_question ):
    # selected_question = been_voted[6:7]
    selected_question = question_content[6:7]
    for i in range(num_question) :
        # selected_question = selected_question.append(been_voted[i*100:i*100+1])
        selected_qid = been_voted[i*100]['question_id']
        selected_content = question_content[ question_content['question_id'] == selected_qid ]
        selected_question = selected_question.append(selected_content)
    return selected_question

In [21]:
# been_voted[6:7].append(been_voted[0:1])
# selected_question = been_voted[6:7]
# selected_question.append(been_voted[0:1])
# Random Select 50 voted Question
selected = Select_Question(49)

In [22]:
# len( selected )
# selected

In [23]:
knn_model = graphlab.nearest_neighbors.create(question_content, features=['tfidf'], label='question_id')

PROGRESS: Starting brute force nearest neighbors model training.


In [24]:
Similar_Qid_List = {}
for i in range( len(selected) ) :
    key = 'Q' + str(i) + '_Similar_List'
    Q_i = selected[i:i+1]
    value = knn_model.query(Q_i, k=30)['reference_label']
    Similar_Qid_List[key] = value

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00322456  | 3.473ms      |
PROGRESS: | Done         |         | 100         | 30.118ms     |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00322456  | 3.167ms      |
PROGRESS: | Done         |         | 100         | 36.381ms     |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | #

In [25]:
def Get_Index_QuestionID(qid) :
    return Total_Test[ Total_Test['question_id'] == qid].index.values

In [26]:
def Get_IndexList(Q_List) :
    IndexList = []
    for qid in Q_List:
        indexes = Get_Index_QuestionID( str(qid) )
        if len(indexes) == 0:
            indexes = Get_Index_QuestionID( qid )
        # print qid, indexes
        for index in indexes:
            IndexList.append(index)
            # print len( IndexList )
    return IndexList

In [27]:
# From Similar Qid List Found Similar Index 
Similar_Index_List = {}
for i in range( len(Similar_Qid_List) ) :
    key = 'Q' + str(i) + '_Similar_List'
    # value is a list of similar qid
    Qid_List = Similar_Qid_List[key]
    Index_List = Get_IndexList(Qid_List)
    Similar_Index_List[key]  = Index_List    

In [28]:
# len( Similar_Index_List )

In [29]:
# import json
# import pickle

def writeDict(dict, filename, sep):
    with open(filename, "w") as f:
#        for key in dict.keys():            
#            f.write(str(key) + ":" + str(dict[key]) + "\n")
        for key, value in sorted( dict.items() ):
            f.write( str(key) + sep + ','.join(str(x) for x in value) + '\n' )

In [30]:
def readDict(filename, sep):
    with open(filename, "r") as f:
        dict = {}
        for line in f:
            values = line.split(sep)
            key = values[0]
            value = values[1].split(',')
            #dict[values[0]] = {int(x) for x in values[1:len(values)]}
            dict[key] = value
        return(dict)

In [31]:
writeDict(Similar_Qid_List, './Similar_Qid_List.txt', ':')
writeDict(Similar_Index_List, './Similar_Index_List.txt', ':')

In [32]:
test = readDict('./Similar_Qid_List.txt', ':')

In [33]:
test == Similar_Qid_List

True